In [15]:
import scvi
import torch
import os
import re
import numpy as np
import pandas as pd
import scanpy as sc
import anndata

In [2]:
# import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.version.cuda)

True
8
11.8


In [3]:
scvi.settings.dl_num_workers = 20
print(f"Changed scvi.settings.dl_num_workers to: {scvi.settings.dl_num_workers}")

Changed scvi.settings.dl_num_workers to: 20


In [16]:
scvi_model_dir = '/home/minhang/mds_project/data/cohort_adata/multiVI_model'
adata_path= '/home/minhang/mds_project/data/cohort_adata/multiVI_model/adata.h5ad'

In [17]:
adata_mvi = sc.read_h5ad(adata_path)
# adata_dummy = sc.read_h5ad(adata_path)

In [22]:
adata_mvi.obsm['ADT'].shape

(192149, 170)

In [19]:
adata_mvi.var['modality'].value_counts()

modality
peaks              298785
Gene Expression     36601
Name: count, dtype: int64

In [7]:
# Ensure the var names are unique if not already
adata_mvi.var_names_make_unique()

# Display feature counts to verify
print("Feature counts by modality:")
if "modality" in adata_mvi.var.columns:
    print(adata_mvi.var["modality"].value_counts())
else:
    print("Warning: adata_mvi.var['modality'] column not found. Ensure features are correctly ordered or n_genes/n_regions/n_proteins are specified correctly.")

print(f"Shape of adata_mvi: {adata_mvi.shape}")
if "ADT" in adata_mvi.obsm_keys():
    print(f"Shape of ADT in obsm: {adata_mvi.obsm['ADT'].shape}")
if "Tech" in adata_mvi.obs.columns:
    print(f"Batch key 'Tech' categories: {adata_mvi.obs['Tech'].unique().tolist()}")
if "sample" in adata_mvi.obs.columns:
    print(f"Categorical covariate 'sample' categories: {adata_mvi.obs['sample'].unique().tolist()}")

Feature counts by modality:
modality
peaks              298785
Gene Expression     36601
Name: count, dtype: int64
Shape of adata_mvi: (192149, 335386)
Shape of ADT in obsm: (192149, 170)
Batch key 'Tech' categories: ['Multi', 'CITE']
Categorical covariate 'sample' categories: ['P01_preSCT', 'P01_Relapse', 'P01_MRD_2', 'P01_Relapse_2', 'P01_preSCT_2', 'P02_Relapse_2', 'P02_preSCT_2', 'P02_Relapse_1', 'P03_MRD_1', 'P02_MRD_1', 'P01_MRD_1', 'P02_preSCT_1', 'P02_Relapse_4', 'P02_Relapse_3', 'P03_Relapse', 'P04_Relapse', 'P04_preSCT_2', 'P04_preSCT_1', 'P05_Relapse', 'P05_preSCT', 'unknown_unknown', 'P06_preSCT_1', 'P06_preSCT_2', 'P10_preSCT', 'P07_Relapse', 'P07_preSCT_2', 'P08_preSCT_2', 'P11_preSCT', 'P08_preSCT_1', 'P08_Relapse', 'P09_preSCT_2', 'P09_preSCT_1', 'P12_preSCT', 'P09_Relapse', 'P03_MRD_2', 'P02_MRD_2', 'P05_MRD', 'P04_MRD', 'P06_MRD', 'P11_MRD', 'P07_MRD', 'P10_MRD', 'P12_MRD', 'P09_MRD', 'P08_MRD', 'P13_MRD', 'P09_MRD_2', 'P09_MRD_3', 'P09_MRD_4', 'P13_Relapse_2', 'P13_p

In [8]:
scvi.model.MULTIVI.setup_anndata(
    adata_mvi,
    batch_key="Tech",  # From collaborator's script
    protein_expression_obsm_key="ADT", # From collaborator's script
    categorical_covariate_keys=["sample"] # From collaborator's script
    # Ensure your adata_mvi.X or specified layers for RNA/ATAC contain raw counts
    # rna_layer="counts", # Or whatever layer has raw RNA counts if .X is not it
    # accessibility_layer="counts", # Or whatever layer has raw ATAC counts
)

INFO     Using column names from columns of adata.obsm['ADT']                                                      


In [9]:
'''
scvi.model.MULTIVI.setup_anndata(
    adata_dummy,
    #batch_key="Tech",  # From collaborator's script
    protein_expression_obsm_key="ADT" # From collaborator's script
    #categorical_covariate_keys=["sample"] # From collaborator's script
    # Ensure your adata_mvi.X or specified layers for RNA/ATAC contain raw counts
    # rna_layer="counts", # Or whatever layer has raw RNA counts if .X is not it
    # accessibility_layer="counts", # Or whatever layer has raw ATAC counts
)
'''

'\nscvi.model.MULTIVI.setup_anndata(\n    adata_dummy,\n    #batch_key="Tech",  # From collaborator\'s script\n    protein_expression_obsm_key="ADT" # From collaborator\'s script\n    #categorical_covariate_keys=["sample"] # From collaborator\'s script\n    # Ensure your adata_mvi.X or specified layers for RNA/ATAC contain raw counts\n    # rna_layer="counts", # Or whatever layer has raw RNA counts if .X is not it\n    # accessibility_layer="counts", # Or whatever layer has raw ATAC counts\n)\n'

In [10]:
# Parameters from your successful CPU load in collaborator's environment
n_genes_val = (adata_mvi.var["modality"] == "Gene Expression").sum() if "modality" in adata_mvi.var.columns else 36601
n_regions_val = (adata_mvi.var["modality"] == "peaks").sum() if "modality" in adata_mvi.var.columns else 298785 # Ensure 'peaks' matches your adata_mvi.var['modality'] value for accessibility
n_proteins_val = adata_mvi.obsm["ADT"].shape[1] if "ADT" in adata_mvi.obsm_keys() else 170 # Inferred from ADT data typically

print(f"Using n_genes: {n_genes_val}, n_regions: {n_regions_val}, n_proteins: {n_proteins_val}")
'''
model_shell = scvi.model.MULTIVI(
    adata_mvi, # adata_mvi already has setup_anndata called on it
    n_genes=n_genes_val,
    n_regions=n_regions_val,
    # n_proteins=n_proteins_val, # REMOVE THIS LINE
    n_hidden=546,
    n_latent=23,
    n_layers_encoder=2,
    n_layers_decoder=2,
    dropout_rate=0.1,
    latent_distribution='normal',
    gene_likelihood='zinb',
    dispersion='gene'
)
'''
model_shell = scvi.model.MULTIVI(
    adata_mvi, # adata_mvi already has setup_anndata called on it
    n_genes=n_genes_val,
    n_regions=n_regions_val,
    # n_proteins=n_proteins_val, # This is fine, it gets inferred if protein_expression_obsm_key is set
    # Ensure these match the training script, if defaults were used, OMIT them:
    # n_hidden=256, # Example: default
    # n_latent=30,  # Example: default
    # n_layers_encoder=1, # Example: default
    # n_layers_decoder=1, # Example: default
    # dropout_rate=0.1, # Or training default
    # latent_distribution='normal', # Or training default
    # gene_likelihood='nb', # Or training default (zinb is also common)
    # dispersion='gene' # Or training default
)

print("MultiVI model shell created with specified parameters.")
print(model_shell) # Verify the parameters

Using n_genes: 36601, n_regions: 298785, n_proteins: 170
MultiVI model shell created with specified parameters.


MultiVI Model with the following params: 
n_genes: 36601, n_regions: 298785, n_proteins: 170, n_hidden: 546, n_latent: 23, n_layers_encoder: 2, 
n_layers_decoder: 2, dropout_rate: 0.1, latent_distribution: normal, deep injection: False, gene_likelihood: zinb, 
gene_dispersion:gene, Mod.Weights: equal, Mod.Penalty: Jeffreys, protein_dispersion: protein
Training status: Not Trained

In [11]:
scvi_model_dir = '/home/minhang/mds_project/data/cohort_adata/multiVI_model'
model_pt_path = os.path.join(scvi_model_dir, 'model.pt')

print(f"Loading file from: {model_pt_path}")
# Load the dictionary, allowing unsafe unpickling
loaded_full_checkpoint = torch.load(model_pt_path, map_location='cpu', weights_only=False)
print("File loaded successfully as a dictionary.")
print("Top-level keys in loaded file:", list(loaded_full_checkpoint.keys()))

# Extract the actual model state dictionary
actual_state_dict = loaded_full_checkpoint['model_state_dict']
print("Extracted 'model_state_dict'. First 5 keys:", list(actual_state_dict.keys())[:5])

# Compare with what your model_shell.module expects
print("First 5 keys from model_shell.module.state_dict():", list(model_shell.module.state_dict().keys())[:5])

# Load the actual state dictionary into the model's module
model_shell.module.load_state_dict(actual_state_dict) # Use the extracted dict
model_shell.is_trained_ = True  # Manually mark the model as trained

print("State_dict loaded into model_shell.module successfully!")
model = model_shell # Now 'model' is your loaded, trained model

# print the loaded_full_checkpoint keys revealing that there are more than just the model_state_dict
print(loaded_full_checkpoint.keys())

Loading file from: /home/minhang/mds_project/data/cohort_adata/multiVI_model/model.pt
File loaded successfully as a dictionary.
Top-level keys in loaded file: ['model_state_dict', 'var_names', 'attr_dict']
Extracted 'model_state_dict'. First 5 keys: ['px_r', 'region_factors', 'background_pro_alpha', 'background_pro_log_beta', 'py_r']
First 5 keys from model_shell.module.state_dict(): ['px_r', 'region_factors', 'background_pro_alpha', 'background_pro_log_beta', 'py_r']
State_dict loaded into model_shell.module successfully!
dict_keys(['model_state_dict', 'var_names', 'attr_dict'])


In [12]:
if torch.cuda.is_available():
    print(f"Moving model to GPU 0...")
    model.to_device("cuda:0") # Or simply "cuda"
    print(f"Model is now on device: {model.device}")
else:
    print("CUDA not available. Model remains on CPU.")

Moving model to GPU 0...
Model is now on device: cuda:0


#### Debugging

In [14]:
# Right before you create temp_loader in your debugging script:
print(f"Model's own device: {model.device}")
print(f"Type of model.device: {type(model.device)}")
if isinstance(model.device, torch.device):
    print(f"Model.device.type: {model.device.type}")
else:
    print("model.device is NOT a torch.device object!")

print(f"Current scvi.settings.dl_num_workers: {scvi.settings.dl_num_workers}")
# ... then create temp_loader and print its pin_memory and tensor devices

Model's own device: cuda:0
Type of model.device: <class 'torch.device'>
Model.device.type: cuda
Current scvi.settings.dl_num_workers: 20


In [15]:
# model is on GPU, model.is_trained_ is True
# THIS IS FOR DEBUGGING ONLY to see if manual movement helps isolate the issue

adata_subset = adata_mvi[np.arange(min(5, adata_mvi.n_obs))].copy() # Small subset

# Re-run setup_anndata on this subset if you are not using model.adata
# scvi.model.MULTIVI.setup_anndata(adata_subset, batch_key="Tech", protein_expression_obsm_key="ADT", categorical_covariate_keys=["sample"])

# Ensure the subset is registered with the model if you pass adata_subset to get_normalized_expression
# Or, just rely on model.adata for the main call.

# For this test, let's mimic the direct data loader usage:
scdl = model._make_data_loader(
    adata=adata_subset, # Or model.adata_manager.adata for a subset of the original
    indices=np.arange(adata_subset.n_obs), # Or indices_for_test
    batch_size=adata_subset.n_obs
)

print("Manually fetching and moving tensors for a test batch:")
for tensors_cpu in scdl: # These will be on CPU
    tensors_gpu = {
        k: v.to(model.device) if isinstance(v, torch.Tensor) else v
        for k, v in tensors_cpu.items()
    }
    print("Manually moved tensors. Devices:")
    for k, v_tensor in tensors_gpu.items():
        if isinstance(v_tensor, torch.Tensor):
            print(f"    Key: {k}, Device: {v_tensor.device}")

    try:
        print("Calling model.module.forward with manually moved GPU tensors...")
        # This is roughly what get_normalized_expression does internally
        with torch.no_grad():
            model.module.eval() # Ensure eval mode
            # Note: _get_generative_input might be needed if directly calling generative
            # We call forward, which then calls inference and generative
            inference_inputs = model.module._get_inference_input(tensors_gpu)
            inference_outputs = model.module.inference(**inference_inputs, n_samples=1) # n_samples from get_normalized_expression default
            generative_inputs = model.module._get_generative_input(tensors_gpu, inference_outputs)
            px = model.module.generative(**generative_inputs) # generative_outputs is px in MULTIVAE

        print("model.module.forward-like calls succeeded with manually moved tensors.")
        # px['px_scale'] would be the normalized expression for the RNA part
        # print(px['px_scale'].shape)
        break # Just do one batch for this test
    except RuntimeError as e:
        print(f"Error even with manually moved tensors: {e}")
        import traceback
        traceback.print_exc()
        break

INFO     AnnData object appears to be a copy. Attempting to transfer setup.                                        
INFO     Found batches with missing protein expression                                                             


Manually fetching and moving tensors for a test batch:


/home/minhang/miniconda3/envs/scvi-env/lib/python3.12/site-packages/scvi/data/fields/_arraylike_field.py:410: UserWarning: Category 4 in adata.obs['sample'] has fewer than 3 cells. Models may not train properly.
  mapping = _make_column_categorical(df, key, key, categorical_dtype=categorical_dtype)


Manually moved tensors. Devices:
    Key: X, Device: cuda:0
    Key: batch, Device: cuda:0
    Key: extra_categorical_covs, Device: cuda:0
    Key: ind_x, Device: cuda:0
    Key: labels, Device: cuda:0
    Key: proteins, Device: cuda:0
Calling model.module.forward with manually moved GPU tensors...
model.module.forward-like calls succeeded with manually moved tensors.


In [13]:
# Debugging block (modified slightly to avoid the AttributeError)
print(f"Model's own device: {model.device}")
print(f"Current scvi.settings.dl_num_workers: {scvi.settings.dl_num_workers}") # Verify it's 2

if model.adata is not None and hasattr(model, 'adata_manager') and model.adata_manager is not None:
    indices_subset = np.arange(min(5, model.adata.n_obs))
    model_adata_manager = model.adata_manager

    num_workers_for_loader = scvi.settings.dl_num_workers # Use the new global setting

    try:
        temp_loader = model._make_data_loader(
            adata=model_adata_manager.adata,
            indices=indices_subset,
            batch_size=len(indices_subset),
            num_workers=num_workers_for_loader
        )
        print(f"Successfully created temp_loader.")
        print(f"  temp_loader.pin_memory (AnnDataLoader's actual pin_memory setting): {temp_loader.pin_memory}") # Should now be True

        test_tensors_batch = next(iter(temp_loader))
        print("\n--- Tensors from AnnDataLoader (first batch) ---")
        for k, v_tensor in test_tensors_batch.items():
            if isinstance(v_tensor, torch.Tensor):
                print(f"    Key: {k}, Device: {v_tensor.device}, Dtype: {v_tensor.dtype}, Shape: {v_tensor.shape}")
        print("--------------------------------------------------\n")

    except Exception as e:
        print(f"Error creating or iterating temp_loader: {e}")
        import traceback
        traceback.print_exc()
else:
    print("model.adata or model.adata_manager is not properly set up for temp_loader check.")

# Then, if the tensors are on GPU, try model.get_normalized_expression(...) again
# ...

Model's own device: cuda:0
Current scvi.settings.dl_num_workers: 2
Successfully created temp_loader.
  temp_loader.pin_memory (AnnDataLoader's actual pin_memory setting): False

--- Tensors from AnnDataLoader (first batch) ---
    Key: X, Device: cpu, Dtype: torch.float32, Shape: torch.Size([5, 335386])
    Key: batch, Device: cpu, Dtype: torch.int64, Shape: torch.Size([5, 1])
    Key: extra_categorical_covs, Device: cpu, Dtype: torch.int64, Shape: torch.Size([5, 1])
    Key: ind_x, Device: cpu, Dtype: torch.int64, Shape: torch.Size([5, 1])
    Key: labels, Device: cpu, Dtype: torch.int64, Shape: torch.Size([5, 1])
    Key: proteins, Device: cpu, Dtype: torch.float32, Shape: torch.Size([5, 170])
--------------------------------------------------



In [ ]:
# Debugging block for temp_loader:
print(f"Model device for DataLoader check: {model.device}")
if model.adata is not None and hasattr(model, 'adata_manager') and model.adata_manager is not None:
    indices_subset = np.arange(min(5, model.adata.n_obs)) # 
    model_adata_manager = model.adata_manager # Use the manager associated with the model instance
    try:
        temp_loader = model._make_data_loader(adata=model_adata_manager.adata, indices=indices_subset, batch_size=len(indices_subset))
        print("Successfully created temp_loader.")
        test_tensors_batch = next(iter(temp_loader))
        print("\n--- Tensors from AnnDataLoader (first batch) ---")
        for k, v_tensor in test_tensors_batch.items():
            if isinstance(v_tensor, torch.Tensor):
                print(f"  Key: {k}, Device: {v_tensor.device}, Dtype: {v_tensor.dtype}, Shape: {v_tensor.shape}")
            else:
                print(f"  Key: {k}, Type: {type(v_tensor)}")
        print("--------------------------------------------------\n")
    except Exception as e:
        print(f"Error creating or iterating temp_loader: {e}")
        import traceback
        traceback.print_exc()
else:
    print("model.adata or model.adata_manager is not properly set up for temp_loader check.")


Model device for DataLoader check: cuda:0
Successfully created temp_loader.

--- Tensors from AnnDataLoader (first batch) ---
  Key: X, Device: cpu, Dtype: torch.float32, Shape: torch.Size([5, 335386])
  Key: batch, Device: cpu, Dtype: torch.int64, Shape: torch.Size([5, 1])
  Key: extra_categorical_covs, Device: cpu, Dtype: torch.int64, Shape: torch.Size([5, 1])
  Key: ind_x, Device: cpu, Dtype: torch.int64, Shape: torch.Size([5, 1])
  Key: labels, Device: cpu, Dtype: torch.int64, Shape: torch.Size([5, 1])
  Key: proteins, Device: cpu, Dtype: torch.float32, Shape: torch.Size([5, 170])
--------------------------------------------------



In [13]:
print(f"Model device for latent space generation: {model.device}")

# Option A: Try the standard method first (might fail due to DataLoader issues)
try:
    print("Attempting model.get_latent_representation()...")
    adata_mvi.obsm['X_multivi'] = model.get_latent_representation(adata_mvi)
    print("Successfully obtained latent representation using model.get_latent_representation().")
    print(f"Shape of X_multivi: {adata_mvi.obsm['X_multivi'].shape}")
except Exception as e:
    print(f"model.get_latent_representation() failed: {e}")
    print("Attempting manual latent space generation loop as a fallback...")

Model device for latent space generation: cuda:0
Attempting model.get_latent_representation()...
model.get_latent_representation() failed: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)
Attempting manual latent space generation loop as a fallback...


In [14]:
print("Attempting to get normalized expression...")
# If your model was saved with an anndata, it might be accessible via model.adata
# If you used adata=None with load, it should have tried to load it from the model_dir.
# When manually constructing, the adata_mvi you used for setup is registered.
norm_exp = model.get_normalized_expression(
    adata=None, # Or use None if the model has adata_mvi registered internally from the constructor
    n_samples=5 # Small number for testing
)
print("Normalized expression retrieved successfully!")
print(norm_exp.shape)
print(norm_exp.head())

Attempting to get normalized expression...


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

#### After debugging

In [13]:
print(f"Using scvi.settings.dl_num_workers: {scvi.settings.dl_num_workers}")

batch_size = 512  # Adjust based on your GPU memory
all_norm_exp_list = []

# Create a DataLoader for the full AnnData object.
# shuffle=False is CRITICAL to maintain original cell order.
full_scdl = model._make_data_loader(
    adata=adata_mvi, # Your full AnnData object used with the model
    shuffle=False,
    batch_size=batch_size,
    # num_workers will be taken from scvi.settings.dl_num_workers
)

print(f"Processing {adata_mvi.n_obs} cells in batches of {batch_size} to get normalized expression...")

Using scvi.settings.dl_num_workers: 2
Processing 192149 cells in batches of 512 to get normalized expression...


In [14]:
with torch.no_grad(): # Ensure no gradients are computed during this inference
    model.module.eval() # Set the underlying nn.Module to evaluation mode

    for tensors_cpu in full_scdl:
        # 1. Manually move tensors to the model's device
        tensors_gpu = {
            k: v.to(model.device) if isinstance(v, torch.Tensor) else v
            for k, v in tensors_cpu.items()
        }

        # 2. Perform inference to get latent representation
        # These kwargs are typical for getting a deterministic latent representation
        inference_kwargs = {"n_samples": 1}
        inference_inputs = model.module._get_inference_input(tensors_gpu)
        inference_outputs = model.module.inference(**inference_inputs, **inference_kwargs)

        # 3. Perform generation of normalized expression
        # These kwargs are typical for what get_normalized_expression would aim for (mean of normalized values)
        generative_kwargs = {"use_z_mean": True}
        generative_inputs = model.module._get_generative_input(tensors_gpu, inference_outputs)
        # The generative output from MULTIVAE contains px, py, pa
        generative_outputs_dict = model.module.generative(**generative_inputs, **generative_kwargs)

        # 4. Extract RNA normalized expression ('px_scale')
        if "px_scale" in generative_outputs_dict:
            batch_norm_exp_gpu = generative_outputs_dict["px_scale"]
            all_norm_exp_list.append(batch_norm_exp_gpu.cpu().numpy()) # Move to CPU and convert to NumPy
        else:
            print(f"Warning: 'px_scale' not found in generative_outputs_dict. Available keys: {generative_outputs_dict.keys()}")
            # Handle this error appropriately, perhaps by breaking or logging
            break

KeyboardInterrupt: 

In [ ]:
# 5. Concatenate results and create DataFrame
if all_norm_exp_list:
    final_norm_exp_np = np.concatenate(all_norm_exp_list, axis=0)

    # Get gene names (ensure this aligns with the model's understanding of genes)
    # In MULTIVI, n_vars usually corresponds to n_genes.
    # The AnnDataManager also stores the column names for the 'X' field if it was set up.
    # model.adata_manager.get_state_registry(REGISTRY_KEYS.X_KEY).column_names
    # However, adata_mvi.var["modality"] is also a reliable source if used for setup.
    gene_expression_mask = (adata_mvi.var["modality"] == "Gene Expression")
    gene_names = adata_mvi.var_names[gene_expression_mask].tolist()

    if final_norm_exp_np.shape[1] == len(gene_names):
        norm_exp_df = pd.DataFrame(
            final_norm_exp_np,
            index=adata_mvi.obs_names, # Assumes cells are in original order due to shuffle=False
            columns=gene_names
        )
        print("\nSuccessfully generated DataFrame for normalized expression:")
        print(norm_exp_df.head())

        # Now you can use norm_exp_df for your downstream analysis
        # For example, save it:
        # norm_exp_df.to_csv("batch_corrected_normalized_expression.csv")

    else:
        print(f"\nShape Mismatch Error: The number of features in the normalized expression output ({final_norm_exp_np.shape[1]})")
        print(f"does not match the number of gene names derived from adata_mvi.var['modality'] ({len(gene_names)}).")
        print(f"Also check against model's n_vars: {model.summary_stats.n_vars if hasattr(model, 'summary_stats') else 'N/A'}")
        print("Please verify the output keys from model.module.generative() and the gene indexing.")
else:
    print("No normalized expression data was generated.")